In [30]:
import keras
import sys
import matplotlib
import numpy as np
import time
import matplotlib.pyplot as plt
import os

sys.path.append('../')

from datasets import CD_Dataset
from models import Unet, MimoNet
from utility import show_batches, from_categorical, train, crop_receptive, predict_full_image
from utility import dice, precision, Pc
from datasets import combine_y_w

seed = int((time.time()*1e6)%1e6)
np.random.seed(seed)
keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=seed)

In [31]:
input_patch_size = [500,500] # input patch (expect output to be smaller)
input_channels = [3] # RGB
output_channels = [2] #cell and bkg
NBATCH = 2 # example patch per batch
EPCS = 1000 # epochs
REG = True # regularization
USEW = True # use weights
W = 10 # importance of weights
JT = True # Just train set means no evaluation during training 

trained_models_path = '../trained_models'
print('\nInput name of net: append type e.g. MIMO_expertiment_name\n')
NAME_NET = raw_input()
if 'MIMO' in NAME_NET :
    MIMO = False
    UNET = True
if 'UNET' in NAME_NET :
    MIMO = True
    UNET = False

NAME_NET = os.path.join(trained_models_path,NAME_NET)


Input name of net: append type e.g. MIMO_expertiment_name

MIMO_exp1


In [32]:
# Load the dataset
dataset_path = '../CD_Dataset'
dataset = CD_Dataset( path=dataset_path, 
                     train_y_path="train_y",  
                     eval_y_path="eval_y", 
                     fit=False, download=True, 
                     num_classes=output_channels[0] )

In [33]:
# define your model
model_input_size = input_patch_size + input_channels
if MIMO:
    model = MimoNet(model_input_size, classes=output_channels[0], regularized=REG)
if UNET:
    model = Unet(model_input_size, classes=output_channels[0], regularized=REG)
model_output_size = list(model.outputs_shape[0])
print("input size: {}\noutput_size: {}".format(model_input_size,model_output_size))

input size: [500, 500, 3]
output_size: [308, 308, 2]


In [34]:
histo = train(model,dataset,n_batch=NBATCH,epochs=EPCS,just_train=JT,use_weights=USEW, W=W, name=NAME_NET)

=========== iteration 1/1000 =============
Epoch 1/1
2/2 [==============================] - 9s 4s/step - loss: 55.8359 - softmax_categorical_crossentropy: 0.7830 - dice_coef: 0.4570
=========== iteration 2/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 503ms/step - loss: 55.5753 - softmax_categorical_crossentropy: 0.7766 - dice_coef: 0.4679
=========== iteration 3/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 503ms/step - loss: 55.2532 - softmax_categorical_crossentropy: 0.7854 - dice_coef: 0.4874
=========== iteration 4/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 504ms/step - loss: 54.8571 - softmax_categorical_crossentropy: 0.7583 - dice_coef: 0.4919
=========== iteration 5/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 501ms/step - loss: 54.4264 - softmax_categorical_crossentropy: 0.7160 - dice_coef: 0.4887
=========== iteration 6/1000 =============
Epoch 1/1
2/2 [====================

2/2 [==============================] - 1s 508ms/step - loss: 43.4160 - softmax_categorical_crossentropy: 0.3352 - dice_coef: 0.7161
=========== iteration 44/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 505ms/step - loss: 43.2497 - softmax_categorical_crossentropy: 0.3587 - dice_coef: 0.7141
=========== iteration 45/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 507ms/step - loss: 43.0195 - softmax_categorical_crossentropy: 0.3158 - dice_coef: 0.7312
=========== iteration 46/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 507ms/step - loss: 42.8101 - softmax_categorical_crossentropy: 0.2913 - dice_coef: 0.7476
=========== iteration 47/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 507ms/step - loss: 42.6269 - softmax_categorical_crossentropy: 0.2906 - dice_coef: 0.7494
=========== iteration 48/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 507ms/step - loss: 42.4580 -

2/2 [==============================] - 1s 510ms/step - loss: 36.4866 - softmax_categorical_crossentropy: 0.0157 - dice_coef: 0.9845
=========== iteration 87/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 512ms/step - loss: 36.4800 - softmax_categorical_crossentropy: 0.1366 - dice_coef: 0.9493
=========== iteration 88/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 507ms/step - loss: 36.2574 - softmax_categorical_crossentropy: 0.0408 - dice_coef: 0.9771
=========== iteration 89/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 507ms/step - loss: 36.1369 - softmax_categorical_crossentropy: 0.0463 - dice_coef: 0.9759
=========== iteration 90/1000 =============
saving model: ../trained_models/MIMO_exp1
Saved model to disk
Epoch 1/1
2/2 [==============================] - 1s 512ms/step - loss: 36.1006 - softmax_categorical_crossentropy: 0.1351 - dice_coef: 0.9507
=========== iteration 91/1000 =============
Epoch 1/1
2/2 [===

2/2 [==============================] - 1s 512ms/step - loss: 31.7357 - softmax_categorical_crossentropy: 0.0112 - dice_coef: 0.9889
=========== iteration 129/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 511ms/step - loss: 31.6759 - softmax_categorical_crossentropy: 0.0515 - dice_coef: 0.9739
=========== iteration 130/1000 =============
saving model: ../trained_models/MIMO_exp1
Saved model to disk
Epoch 1/1
2/2 [==============================] - 1s 506ms/step - loss: 31.5376 - softmax_categorical_crossentropy: 0.0129 - dice_coef: 0.9876
=========== iteration 131/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 510ms/step - loss: 31.4442 - softmax_categorical_crossentropy: 0.0186 - dice_coef: 0.9882
=========== iteration 132/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 507ms/step - loss: 31.3716 - softmax_categorical_crossentropy: 0.0446 - dice_coef: 0.9773
=========== iteration 133/1000 =============
Epoch 1/1
2/2

2/2 [==============================] - 1s 510ms/step - loss: 27.9527 - softmax_categorical_crossentropy: 0.0574 - dice_coef: 0.9691
=========== iteration 171/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 508ms/step - loss: 27.8227 - softmax_categorical_crossentropy: 0.0102 - dice_coef: 0.9899
=========== iteration 172/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 514ms/step - loss: 27.7587 - softmax_categorical_crossentropy: 0.0286 - dice_coef: 0.9948
=========== iteration 173/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 512ms/step - loss: 27.7043 - softmax_categorical_crossentropy: 0.0563 - dice_coef: 0.9735
=========== iteration 174/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 511ms/step - loss: 27.6562 - softmax_categorical_crossentropy: 0.0900 - dice_coef: 0.9673
=========== iteration 175/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 517ms/step - loss: 27.5

...

2/2 [==============================] - 1s 509ms/step - loss: 6.1136 - softmax_categorical_crossentropy: 0.0476 - dice_coef: 0.9752
=========== iteration 762/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 513ms/step - loss: 6.1675 - softmax_categorical_crossentropy: 0.1158 - dice_coef: 0.9581
=========== iteration 763/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 514ms/step - loss: 6.0685 - softmax_categorical_crossentropy: 0.0310 - dice_coef: 0.9776
=========== iteration 764/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 510ms/step - loss: 6.1558 - softmax_categorical_crossentropy: 0.1326 - dice_coef: 0.9643
=========== iteration 765/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 512ms/step - loss: 6.0300 - softmax_categorical_crossentropy: 0.0209 - dice_coef: 0.9877
=========== iteration 766/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 515ms/step - loss: 6.1228 - 

2/2 [==============================] - 1s 512ms/step - loss: 5.6306 - softmax_categorical_crossentropy: 0.1450 - dice_coef: 0.9600
=========== iteration 805/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 517ms/step - loss: 5.4812 - softmax_categorical_crossentropy: 0.0084 - dice_coef: 0.9916
=========== iteration 806/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 512ms/step - loss: 5.4692 - softmax_categorical_crossentropy: 0.0092 - dice_coef: 0.9909
=========== iteration 807/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 513ms/step - loss: 5.5002 - softmax_categorical_crossentropy: 0.0529 - dice_coef: 0.9699
=========== iteration 808/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 515ms/step - loss: 5.4888 - softmax_categorical_crossentropy: 0.0541 - dice_coef: 0.9976
=========== iteration 809/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 511ms/step - loss: 5.4455 - 

2/2 [==============================] - 1s 514ms/step - loss: 5.0093 - softmax_categorical_crossentropy: 0.0416 - dice_coef: 0.9788
=========== iteration 848/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 510ms/step - loss: 4.9648 - softmax_categorical_crossentropy: 0.0085 - dice_coef: 0.9921
=========== iteration 849/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 509ms/step - loss: 4.9518 - softmax_categorical_crossentropy: 0.0068 - dice_coef: 0.9933
=========== iteration 850/1000 =============
saving model: ../trained_models/MIMO_exp1
Saved model to disk
Epoch 1/1
2/2 [==============================] - 1s 513ms/step - loss: 5.0345 - softmax_categorical_crossentropy: 0.1008 - dice_coef: 0.9667
=========== iteration 851/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 517ms/step - loss: 4.9407 - softmax_categorical_crossentropy: 0.0182 - dice_coef: 0.9841
=========== iteration 852/1000 =============
Epoch 1/1
2/2 [===

Saved model to disk
Epoch 1/1
2/2 [==============================] - 1s 511ms/step - loss: 4.5241 - softmax_categorical_crossentropy: 0.0200 - dice_coef: 0.9841
=========== iteration 891/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 511ms/step - loss: 4.5192 - softmax_categorical_crossentropy: 0.0253 - dice_coef: 0.9855
=========== iteration 892/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 508ms/step - loss: 4.4924 - softmax_categorical_crossentropy: 0.0086 - dice_coef: 0.9943
=========== iteration 893/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 510ms/step - loss: 4.4965 - softmax_categorical_crossentropy: 0.0228 - dice_coef: 0.9861
=========== iteration 894/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 511ms/step - loss: 4.5561 - softmax_categorical_crossentropy: 0.0925 - dice_coef: 0.9639
=========== iteration 895/1000 =============
Epoch 1/1
1/2 [==============>...............] - E

KeyboardInterrupt: 